In [12]:
import os
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# Step 1: Load and Preprocess the Data
def load_images_and_labels(data_dir):
    images = []
    labels = []
    for disease_folder in os.listdir(data_dir):
        if os.path.isdir(os.path.join(data_dir, disease_folder)):
            for filename in os.listdir(os.path.join(data_dir, disease_folder)):
                img_path = os.path.join(data_dir, disease_folder, filename)
                
                # Check if the file is an image
                if not img_path.endswith(('.jpg', '.jpeg', '.png')):
                    continue

                # Load and preprocess the image
                img = cv2.imread(img_path)

                # Check if the image is loaded successfully
                if img is None:
                    print(f"Warning: Unable to load image {img_path}")
                    continue

                # Resize images to 224x224
                img = cv2.resize(img, (224, 224))
                
                # Normalize pixel values
                img = img / 255.0

                images.append(img)
                labels.append(disease_folder)
    return np.array(images), np.array(labels)

# Step 2: Label Encoding
def encode_labels(labels):
    unique_labels = list(set(labels))
    label_dict = {label: i for i, label in enumerate(unique_labels)}
    encoded_labels = [label_dict[label] for label in labels]
    return np.array(encoded_labels), label_dict

# Step 3: Feature Extraction
def extract_features(images):
    base_model = VGG16(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    feature_extractor = Model(inputs=base_model.input, outputs=x)

    features = feature_extractor.predict(images)
    return features

# Specify the path to your dataset
data_dir = 'Dataset'

# Load and preprocess data
images, labels = load_images_and_labels(data_dir)

# Encode labels
encoded_labels, label_dict = encode_labels(labels)

# Extract features
features = extract_features(images)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Step 4: Train the SVM Model
svm_model = svm.SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, y_pred))


11/11 [==============================] - 48s 4s/step
Accuracy: 0.8823529411764706
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91        11
           1       1.00      0.80      0.89         5
           2       0.25      0.50      0.33         2
           3       1.00      1.00      1.00         6
           4       0.00      0.00      0.00         1
           5       0.86      0.86      0.86        14
           6       0.91      0.91      0.91        11
           7       1.00      0.80      0.89         5
           8       0.93      1.00      0.96        13

    accuracy                           0.88        68
   macro avg       0.76      0.75      0.75        68
weighted avg       0.89      0.88      0.88        68



c:\Users\heman\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heman\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\heman\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [19]:

# Create label_dict_inv after label_dict is created
label_dict_inv = {v: k for k, v in label_dict.items()}

# Step 6: Make prediction on a single image
def preprocess_single_image(img_path):
    img = cv2.imread(img_path)

    # Check if the image is loaded successfully
    if img is None:
        print(f"Error: Unable to load image {img_path}")
        return None

    # Resize image to 224x224
    img = cv2.resize(img, (224, 224))

    # Normalize pixel values
    img = img / 255.0

    return np.expand_dims(img, axis=0)  # Add batch dimension

# Extract features from a single image
def extract_features_single_image(img):
    base_model = VGG16(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    feature_extractor = Model(inputs=base_model.input, outputs=x)
    features = feature_extractor.predict(img)
    return features

# Make prediction on a single image
def predict_single_image(img_path):
    # Preprocess the image
    img = preprocess_single_image(img_path)

    if img is None:
        return None

    # Extract features
    img_features = extract_features_single_image(img)

    # Make prediction using the trained SVM model
    prediction = svm_model.predict(img_features)

    # Map numerical prediction back to class label
    predicted_label = label_dict_inv[prediction[0]]

    return predicted_label

# Specify the path to the image you want to predict
image_path_to_predict = 'img3.jpeg'

# Make prediction
predicted_disease = predict_single_image(image_path_to_predict)

if predicted_disease is not None:
    print(f"The predicted disease for the given image is: {predicted_disease}")


1/1 [==============================] - 0s 391ms/step
The predicted disease for the given image is: Bacterial blight


In [20]:
import joblib


# Step 5: Train the SVM Model
svm_model = svm.SVC(kernel='linear', C=1.0)
svm_model.fit(X_train, y_train)

# Save the trained SVM model to a joblib file
joblib.dump(svm_model, 'svm_model.joblib')




['svm_model.joblib']